In [91]:
import requests
import pprint
import pandas as pd

pp = pprint.PrettyPrinter(indent=4)


def get_recipes():
    _app_id = '844ee8f7'
    _app_key = '9b846490c7c34c4f33e70564831f232b'

    all_recipes = requests.get('http://api.yummly.com/v1/api/recipes',
                               params={'_app_id':_app_id,
                                       '_app_key':_app_key,
                                       'q':'chicken',
                                       'maxResult':40})
    all_recipes = all_recipes.json()
    return all_recipes['matches']

def recipe_ingr_parse(recipe_list):
    recipe_ing = [(recipe['id'], ingredient) for recipe in recipe_list for ingredient in recipe['ingredients']]
    return recipe_ing

def import_ingre_fc():
    #will want this to be done once and stored in db
    fc_indr_raw = pd.read_csv('master_fc_ing.csv')
    fc_indr_raw['ingredient_name'] = [ingredient.replace('_', ' ') for ingredient in fc_indr_raw['ingredient_name']]
    fc_ingr_only['matched'] = fc_ingr_only[['ingredient_name', 'compound_id']].apply(tuple, axis=1)
    ingredients_fc_id = {ingredient: [] for ingredient in fc_ingr_only['ingredient_name']}
    for ingredient, fc_id in list(fc_ingr_only['matched']):
        if ingredient in ingredients_fc_id:
                ingredients_fc_id[ingredient].append(fc_id)
    return ingredients_fc_id


fc_ingr_only_dict = import_ingre_fc()
def fc_return(recipe_id, ingredient):
    # will need to look in db table long term instead of dict.
    return [(recipe_id, fc_id, ingredient) for fc_id in fc_ingr_only_dict[ingredient]]

def recipe_fc_total_lookup(recipe_ingr_list):
    recipe_fc_final = []
    for recipe_id, ingredient in recipe_ing:
        if ingredient in fc_ingr_only_dict.keys():
            recipe_fc_final.extend(fc_return(recipe_id, ingredient))
    return recipe_fc_final

In [92]:
import timeit
fc_ingr_only_dict = import_ingre_fc()

start = timeit.default_timer()

yum_recipes = get_recipes()
recipe_ing = recipe_ingr_parse(yum_recipes)

recipe_fc_final = []
for recipe_id, ingredient in recipe_ing:
    if ingredient in fc_ingr_only_dict.keys():
        recipe_fc_final.extend(fc_return(recipe_id, ingredient))
    
stop = timeit.default_timer()
print(stop - start, ' seconds')

0.6835767620068509  seconds


In [93]:
recipe_fc_final

[('Man-Pleasing-Chicken-1010619', 748, 'maple syrup'),
 ('Man-Pleasing-Chicken-1010619', 876, 'maple syrup'),
 ('Man-Pleasing-Chicken-1010619', 879, 'maple syrup'),
 ('Man-Pleasing-Chicken-1010619', 37, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 79, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 132, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 165, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 177, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 243, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 250, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 260, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 273, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 295, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 348, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 361, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 378, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 393, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 434, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 442, 'pepper'),
 ('Man-Pleasing-Chicken-1010619', 460

In [88]:
recipe_ing

[('Man-Pleasing-Chicken-1010619', 'dijon mustard'),
 ('Man-Pleasing-Chicken-1010619', 'maple syrup'),
 ('Man-Pleasing-Chicken-1010619', 'rice vinegar'),
 ('Man-Pleasing-Chicken-1010619', 'chicken thighs'),
 ('Man-Pleasing-Chicken-1010619', 'salt'),
 ('Man-Pleasing-Chicken-1010619', 'pepper'),
 ('Balsamic-Glazed-Chicken-Breast-1036296',
  'boneless skinless chicken breasts'),
 ('Balsamic-Glazed-Chicken-Breast-1036296', 'extra-virgin olive oil'),
 ('Balsamic-Glazed-Chicken-Breast-1036296', 'balsamic vinegar'),
 ('Roast-Chicken-with-Lemon-1024103', 'chicken'),
 ('Roast-Chicken-with-Lemon-1024103', 'olive oil'),
 ('Roast-Chicken-with-Lemon-1024103', 'kosher salt'),
 ('Roast-Chicken-with-Lemon-1024103', 'ground black pepper'),
 ('Roast-Chicken-with-Lemon-1024103', 'lemon'),
 ('Crockpot-Honey-Rosemary-Chicken-1037334', 'chicken breasts'),
 ('Crockpot-Honey-Rosemary-Chicken-1037334', 'balsamic vinegar'),
 ('Crockpot-Honey-Rosemary-Chicken-1037334', 'honey'),
 ('Crockpot-Honey-Rosemary-Chicken